# Evaluating Docugami KG-RAG against OpenAI Assistants Retrieval

## Set up Eval Dataset

In [ ]:
import os
from pathlib import Path

DOCSET_NAME = "Earnings Calls Evaluation"
FILE_NAMES = [
    "Q1 2022 Snowflake Inc. Earnings Call - Snowflake Inc - BamSEC.pdf",
    "Q1 2023 Snowflake Inc. Earnings Call - Snowflake Inc - BamSEC.pdf",
    "Q2 2022 Snowflake Inc. Earnings Call - Snowflake Inc - BamSEC.pdf",
    "Q2 2023 Snowflake Inc. Earnings Call - Snowflake Inc - BamSEC.pdf",
    "Q3 2021 Snowflake Inc. Earnings Call - Snowflake Inc - BamSEC.pdf",
    "Q3 2022 Snowflake Inc. Earnings Call - Snowflake Inc - BamSEC.pdf",
    "Q3 2023 Snowflake Inc. Earnings Call - Snowflake Inc - BamSEC.pdf",
    "Q4 2020 Snowflake Inc. Earnings Call - Snowflake Inc - BamSEC.pdf",
    "Q4 2022 Snowflake Inc. Earnings Call - Snowflake Inc - BamSEC.pdf",
    "Q3 FY23 Microsoft Corp Earnings Call.pdf",
]

FILES_DIR = Path(os.getcwd()) / "v1/docs"
GROUND_TRUTH_CSV = Path(os.getcwd()) / "v1/ground-truth-earning_calls.csv"

# Note: Please specify ~6 (or more!) similar files to process together as a document set
# This is currently a requirement for Docugami to automatically detect motifs
# across the document set to generate a semantic XML Knowledge Graph.
assert len(FILE_NAMES) >= 6, "Please provide at least 6 files"

In [ ]:
import pandas as pd
from langsmith import Client

# Read
df = pd.read_csv(GROUND_TRUTH_CSV)

# Dataset
client = Client()
dataset_name = DOCSET_NAME
existing_datasets = list(client.list_datasets(dataset_name=dataset_name))
if existing_datasets:
    # read existing dataset
    dataset = client.read_dataset(dataset_name=dataset_name)
else:
    dataset = client.create_dataset(dataset_name=dataset_name)
    # Populate dataset
    for _, row in df.iterrows():
        q = row["Question"]
        a = row["Answer"]
        client.create_example(
            inputs={"question": q}, outputs={"answer": a}, dataset_id=dataset.id
        )

## Set up Docugami KG-RAG

In [ ]:
! pip install pip --quiet --upgrade
! pip install docugami==0.0.9 dgml-utils==0.3.0 --quiet --upgrade

#### Upload files to Docugami

In [ ]:
from docugami import Docugami
from docugami.lib.upload import upload_to_named_docset, wait_for_dgml

dg_client = Docugami()
file_paths = [FILES_DIR / file_name for file_name in FILE_NAMES]

# Files will not be re-uploaded if they were previously uploaded (based on name)
dg_docs = upload_to_named_docset(dg_client, file_paths, DOCSET_NAME)

docset_id = ""
docset_name = ""
for doc in dg_docs:
    if not docset_id:
        docset_id = doc.docset.id
    else:
        # all docs must be in the same docset
        assert docset_id == doc.docset.id

    if not docset_name:
        docset_name = dg_client.docsets.retrieve(doc.docset.id).name

In [ ]:
# Wait for files to finish processing (OCR, and zero-shot creation of XML knowledge graph)

# Note: This can take some time on the free docugami tier (up to ~20 mins). Please contact us for faster paid plans.
wait_for_dgml(dg_client, dg_docs)

In [ ]:
# Run indexing
from docugami_kg_rag.helpers.indexing import index_docset

assert docset_id
assert docset_name

# Note: This can take some time since it is embedding and creating summaries for all the docs and chunks
index_docset(docset_id=docset_id, name=docset_name)

#### Create Docugami Agent

In [ ]:
from langchain.agents import AgentExecutor
from docugami_kg_rag.chain import agent as docugami_agent, _get_tools, AgentInput

def predict_docugami_agent(input: dict) -> dict:
    question = input["question"]
    chain = AgentExecutor(
        agent=docugami_agent,
        tools=_get_tools(),
    ).with_types(
        input_type=AgentInput,
    )
    result = chain.invoke({
        "input": question,
        "use_reports": False,
        "chat_history": [],
    })

    return result["output"]

In [ ]:
# Test the agent to make sure it is working
predict_docugami_agent({"question": "What was the question from Barclays in the Q2 2023 earnings call?"})

## Set up OpenAI Assistants Retrieval

In [ ]:
! pip install openai --upgrade --quiet

### Upload files to OpenAI

In [ ]:
from openai import files

existing_files = files.list().data
existing_file_names = [f.filename for f in existing_files]
uploaded_files = []
for file_name in FILE_NAMES:
    if file_name in existing_file_names:
        # file was previously uploaded
        uploaded_files.append([f for f in existing_files if f.filename == file_name][0])
    else:
        # upload
        file_path = FILES_DIR / file_name
        file = files.create(file=file_path, purpose='assistants')
        uploaded_files.append(file)

file_ids=[f.id for f in uploaded_files]

uploaded_files



### Create OpenAI Assistant Agent

In [ ]:
from langchain.agents.openai_assistant import OpenAIAssistantRunnable
openai_agent = OpenAIAssistantRunnable.create_assistant(
    name="Earnings Call Assistant",
    instructions="An assistant that specializes in answering questions based only on the given knowledge base of earnings calls",
    tools=[{"type": "retrieval"}],
    model="gpt-4-1106-preview",
)
openai_agent.as_agent = True

def predict_openai_agent(input: dict) -> dict:
    question = input["question"]
    result = openai_agent.invoke({
        "content": question,
        "file_ids": file_ids,
    })

    return result.return_values["output"]

In [ ]:
# Test the agent to make sure it is working
predict_openai_agent({"question": "What was the question from Barclays in the Q2 2023 earnings call?"})

## Run Evals


In [ ]:
import uuid
from langsmith.client import Client
from langchain.smith import RunEvalConfig

eval_config = RunEvalConfig(
    evaluators=["cot_qa"],
)

def run_eval(eval_func, eval_run_name):
    """
    Run eval
    """
    client = Client()
    client.run_on_dataset(
        dataset_name=DOCSET_NAME,
        llm_or_chain_factory=eval_func,
        evaluation=eval_config,
        verbose=True,
        project_name=eval_run_name,
    )


# Experiments
agent_map = {
    "docugami_kg_rag_zero_shot": predict_docugami_agent,
    "openai_assistant_retrieval": predict_openai_agent,
}

for _ in range(10):
    run_id = str(uuid.uuid4())
    for project_name, agent in agent_map.items():
        run_eval(agent, project_name + "_" + run_id)